# Chapter 4 신경망 학습  
**학습이란?**    
  
훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것  
손실 함수의 결괏값을 최소로 만드는 가중치 매개변수를 찾는 것이 학습의 목표

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### 4.2.1 오차제곱합  
$ E = \frac{1}{2}\sum_{k}(y_k - t_k)^2 $  
  
여기서 $\frac{1}{2}$이 곱해진 이유는 **델타 규칙(Delta Rule)** 때문인데, 최적의 가중치를 찾아가는 최적화(Optimizer)에서 사용되는 경사하강법은 기울기를 기반으로 실시되며, 이 과정에서 발생할 수 있는 오류를 최소화시키기 위해서이다.

In [2]:
def sum_square_error(y, t):
    return 0.5 * np.sum((y - t) ** 2)

In [3]:
# 정답은 '2'
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# ex1) '2'일 확률이 가장 높다고 추정함 (0.6)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
sum_square_error(np.array(y), np.array(t))

0.09750000000000003

In [4]:
# ex2) '7'일 확률이 가장 높다고 추정함 (0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
sum_square_error(np.array(y), np.array(t))

0.5975

첫 번째 예시가 두 번째 예시보다 손실 함수 값이 더 작으므로 더 정답에 가까움을 알 수 있습니다.

### 4.2.2 교차 엔트로피 오차  
$ E = -\sum_{k}{t_k}\log{y_k}$

In [5]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

In [6]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [7]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

### 4.2.3 미니배치 학습

크기가 매우 큰 데이터를 가지고 일일이 손실 함수를 계산하는 것은 비효율적입니다.  
이런 경우 데이터의 일부만 사용해 전체의 '근사치'를 구할 수 있습니다.  
이 일부를 **미니 배치** 라고 합니다. 

In [8]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

In [9]:
(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)
    
print(x_train.shape)
print(t_train.shape)
t_train.size

(60000, 784)
(60000, 10)


600000

In [10]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

### 4.2.4 (배치용) 교차 엔트로피 오차 구하기

In [11]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size) # size = 행 x 열 크기
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [12]:
# 정답 레이블이 one-hot 인코딩이 아닐 떄 교차 엔트로피 오차 구하기
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size # 이 내용 한 번 더 보고 이해를 확실히 할 필요가 있음